In [23]:
# Instalación de dependencias (silenciosa)
%pip install langchain langchain-openai python-dotenv openai -q

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import ast

load_dotenv()

True

In [25]:
def crear_cadena(llm, input_variables, template):
    prompt = PromptTemplate(input_variables=input_variables, template=template)
    return LLMChain(llm=llm, prompt=prompt)

def convertir_a_diccionario(texto_llm):
    """
    Intenta convertir una cadena de texto generada por el modelo en un diccionario de Python válido.
    Si el texto viene en formato Markdown (ej: con etiquetas ```python), las limpia antes de evaluarlo.
    """
    if texto_llm.startswith("```"):
        texto_llm = texto_llm.strip("`").strip()
        if texto_llm.startswith("python"):
            texto_llm = texto_llm[len("python"):].strip()

    try:
        return ast.literal_eval(texto_llm)
    except Exception as e:
        print("❌ Error al interpretar la respuesta como diccionario:", e)
        print("🧾 Contenido recibido:", texto_llm)
        exit()

In [26]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Paso 1: Extraer información

In [27]:
extract_chain = crear_cadena(llm, ["email"], """A partir del siguiente correo extrae esta información:
- Número de pedido
- Nombre del remitente
- Motivo principal de la solicitud

Email:
\"\"\"{email}\"\"\"

Devuelve un diccionario de Python con las claves: pedido, remitente y motivo. 
No utilices formato Markdown ni encierres el texto entre comillas triples ni uses etiquetas como '```python'.
""")

# Paso 2: Evaluar

In [28]:
eval_chain = crear_cadena(llm, ["motivo"], """Según este motivo: "{motivo}", indica si se debe ACEPTAR o RECHAZAR la devolución.

✅ ACEPTAR si:
- Defecto de fabricación
- Error en el suministro
- Producto incompleto desde fábrica

❌ RECHAZAR si:
- Daños durante el transporte (si no es responsabilidad de la empresa)
- Manipulación del cliente
- Solicitud fuera de plazo

Responde con: "ACEPTAR" o "RECHAZAR"
""")

# Paso 3: Redactar respuesta con firma personalizada

In [29]:
response_chain = crear_cadena(llm, 
    ["decision", "remitente", "pedido", "nombre", "cargo", "empresa", "contacto"], 
    """Redacta una respuesta formal y empática para el cliente {remitente}, sobre el pedido {pedido}.

Si la decisión es ACEPTAR:
- Agradécele por contactar.
- Confirma que el reemplazo será procesado.
- Explica brevemente que la empresa cubrirá el fallo según la política.

Si la decisión es RECHAZAR:
- Lamenta la situación.
- Explica que no se puede aceptar la devolución según la política.
- Muestra comprensión y ofrece ayuda adicional si la necesita.

Finaliza con esta firma profesional:

{nombre}
{cargo}
{empresa}
{contacto}

Decisión: {decision}
""")

# Ejecutar pasos

# CASO 1: Solicitud que debe ser RECHAZADA

In [30]:
email = """Asunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA

Estimado equipo de Componentes Intergalácticos Industriales S.A.,

Me pongo en contacto con ustedes como cliente reciente para comunicar una incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de condensadores de fluzo modelo FX-88.

Lamentablemente, al recibir el envío, observamos que varios de los condensadores presentaban daños visibles. Todo indica que la mercancía sufrió una caída durante el transporte interestelar.

Solicitamos con urgencia el reemplazo inmediato de las unidades defectuosas.

Atentamente,
Darth Márquez
"""

# Ejecutar paso 1
extract_result = extract_chain.invoke(email)
info = convertir_a_diccionario(extract_result['text'])

pedido = info["pedido"]
remitente = info["remitente"]
motivo = info["motivo"]

# Ejecutar paso 2
decision_raw = eval_chain.invoke({"motivo": motivo})
decision = decision_raw["text"].strip()

# Ejecutar paso 3

# Datos de firma personalizados
firma_info = {
    "nombre": "María Fernández",
    "cargo": "Responsable de Atención al Cliente",
    "empresa": "Componentes Intergalácticos Industriales S.A.",
    "contacto": "contacto@cii.com"
}

response_result = response_chain.invoke({
    "decision": decision,
    "remitente": remitente,
    "pedido": pedido,
    **firma_info # Con ** copìas todo lo de firma_info
})

print(response_result["text"])

Estimado Darth Márquez,

Gracias por ponerse en contacto con nosotros respecto a su pedido #D347-STELLA. Lamentamos sinceramente la situación que ha experimentado con su producto.

Después de revisar su caso detenidamente, lamentamos informarle que no podemos aceptar la devolución del producto, ya que no cumple con los criterios establecidos en nuestra política de devoluciones. Entendemos que esto puede ser decepcionante y queremos asegurarle que valoramos su comprensión en este asunto.

Estamos aquí para ofrecerle cualquier asistencia adicional que pueda necesitar. No dude en ponerse en contacto con nosotros si tiene alguna otra consulta o si podemos ayudarle de alguna otra manera.

Atentamente,

María Fernández  
Responsable de Atención al Cliente  
Componentes Intergalácticos Industriales S.A.  
contacto@cii.com


# CASO 2: Solicitud que debe ser ACEPTADA

In [31]:
email_aceptar = """Asunto: Devolución por defecto de fabricación – Pedido #XZ901-LUCA

Estimado equipo de Componentes Intergalácticos Industriales S.A.,

Les escribo para informarles que el pedido #XZ901-LUCA, compuesto por una serie de microprocesadores CU-92, presenta un defecto de fábrica: varias unidades no responden a la activación básica ni siquiera tras revisión técnica.

Solicito formalmente la devolución o el reemplazo de las unidades defectuosas.

Gracias por su atención.

Atentamente,
Lucía Robles
"""

# Ejecutar paso 1
extract_result_aceptar = extract_chain.invoke(email_aceptar)
info_aceptar = convertir_a_diccionario(extract_result_aceptar['text'])

pedido2 = info_aceptar["pedido"]
remitente2 = info_aceptar["remitente"]
motivo2 = info_aceptar["motivo"]

# Ejecutar paso 2
decision_raw2 = eval_chain.invoke({"motivo": motivo2})
decision2 = decision_raw2["text"].strip()

# Ejecutar paso 3
response_result2 = response_chain.invoke({
    "decision": decision2,
    "remitente": remitente2,
    "pedido": pedido2,
    **firma_info
})

print(response_result2["text"])

Estimada Lucía Robles,

Gracias por ponerse en contacto con nosotros y por informarnos sobre el inconveniente con su pedido XZ901-LUCA. Lamentamos mucho cualquier inconveniente que esto haya podido causarle.

Nos complace informarle que hemos revisado su caso y el reemplazo de su pedido será procesado de acuerdo con nuestra política de garantía. Nos aseguraremos de que el fallo sea cubierto completamente por nuestra empresa, ya que valoramos su satisfacción y confianza en nuestros productos.

Si tiene alguna pregunta adicional o necesita más asistencia, no dude en ponerse en contacto con nosotros. Estamos aquí para ayudarle en lo que necesite.

Atentamente,

María Fernández  
Responsable de Atención al Cliente  
Componentes Intergalácticos Industriales S.A.  
contacto@cii.com
